In [1]:
import os
import os.path as op
import nibabel as nib
import numpy as np

from dipy.io.streamline import load_trk
from dipy.tracking.streamline import transform_streamlines

from fury import actor, window
from fury.colormap import create_colormap

import AFQ.data.fetch as afd

import boto3
from botocore import UNSIGNED
from botocore.client import Config
from tqdm import tqdm

import os
import os.path as op
import json
from glob import glob
import shutil
import pandas as pd

# import statsmodels.formula.api as sm
# from statsmodels.stats.multitest import fdrcorrection
import seaborn as sns
from matplotlib import pyplot as plt
import math


/Users/Ethan/opt/anaconda3/envs/pyafq_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
## Helper function to convert 30-color hex palette to RGB

hex_list = ["#fcff5d","#7dfc00","#0ec434","#228c68","#8ad8e8","#235b54","#29bdab","#3998f5",
            "#37294f","#946aa2","#3750db","#f22020","#991919","#ffcba5","#e68f66","#c56133",
            "#96341c","#632819","#ffc413","#f47a22","#2f2aa0","#b732cc","#772b9d","#f07cab",
            "#d30b94","#edeff3","#946aa2","#5d4c86",]

def hex_to_rgb(hex_value):
    hex_value = hex_value.lstrip("#")
    return tuple(int(hex_value[i:i+2], 16) for i in (0, 2, 4))

rgb_list = [hex_to_rgb(hex_value) for hex_value in hex_list]


## Load Model Results

In [38]:
# Load mixed-effects model output
model_output_df = pd.read_csv('/Users/Ethan/Documents/Stanford/afq/abcd/data/baseline_fa_model_output_updated_ses_no_qsi_prep_fa_int.csv')
model_output_df['neg_log10p_adj'] = -1* np.log(model_output_df['p_adj'])



/Users/Ethan/opt/anaconda3/envs/pyafq_env/lib/python3.9/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


## Generate Visualizations

### Helper functions for generating bundle renderings

In [89]:
if os.environ.get("XVFB", False):
    print("Initializing XVFB")
    import xvfbwrapper
    from xvfbwrapper import Xvfb

    vdisplay = Xvfb()
    vdisplay.start()

In [90]:
def lines_as_tubes(sl, line_width, **kwargs):
    line_actor = actor.line(sl, **kwargs)
    line_actor.GetProperty().SetRenderLinesAsTubes(1)
    line_actor.GetProperty().SetLineWidth(line_width)
    return line_actor


    

In [91]:
def slice_volume(data, x=None, y=None, z=None):
    slicer_actors = []
    slicer_actor_z = actor.slicer(data)
    if z is not None:
        slicer_actor_z.display_extent(
            0, data.shape[0] - 1,
            0, data.shape[1] - 1,
            z, z)
        slicer_actors.append(slicer_actor_z)
    if y is not None:
        slicer_actor_y = slicer_actor_z.copy()
        slicer_actor_y.display_extent(
            0, data.shape[0] - 1,
            y, y,
            0, data.shape[2] - 1)
        slicer_actors.append(slicer_actor_y)
    if x is not None:
        slicer_actor_x = slicer_actor_z.copy()
        slicer_actor_x.display_extent(
            x, x,
            0, data.shape[1] - 1,
            0, data.shape[2] - 1)
        slicer_actors.append(slicer_actor_x)

    return slicer_actors


slicers = slice_volume(t1w, x=t1w.shape[0]//2)
slicers_z = slice_volume(t1w, z=t1w.shape[-1] // 3)

### Grab Example Bundles + T1w Image

In [60]:
## update paths and file names to reflect your directory structure

afq_path = '/path/to/afq/outputs/bundle_renderings'
bundle_path = '/path/to/afq/outputs/bundle_renderings/clean_bundles'


In [61]:
## Load FA image for exemplar participant

fa_img = nib.load(op.join(afq_path,
'sub-EXAMPLE_SUB-ses-EXAMPLE_SES_space-T1w_desc-preproc_dwi_model-DKI_desc-FA_dwi.nii.gz'))
fa = fa_img.get_fdata()



In [62]:
## Load tractography for each bundle in exemplar participant

tracts = ['ARCL', 'ARCR', 'ATRL', 'ATRR', 'AntFrontal', 'CGCL', 'CGCR',
       'CSTL', 'CSTR','IFOL', 'IFOR', 'ILFL', 'ILFR',
       'Motor', 'Occipital', 'Orbital', 'PostParietal', 'SLFL', 'SLFR',
       'SupFrontal', 'SupParietal', 'Temporal', 'UNCL', 'UNCR','pARCL','pARCR','VOFL','VOFR']

bundle_dict = {
    tract: {'tractogram':load_trk(op.join(bundle_path,
     'sub-EXAMPLE_SUB-ses-EXAMPLE_SES_space-T1w_desc-preproc_dwi_space-RASMM_model-probCSD_algo-AFQ_desc-'+tract+'_tractography.trk'),
                                 fa_img)}
    for tract in tracts
}

In [63]:
# Load t1 image for exemplar subject

t1w_img = nib.load(op.join(deriv_path,
'qsiprep/sub-EXAMPLE_SUB/ses-EXAMPLE-SES/anat/sub-EXAMPLE_SUB-ses-EXAMPLE_SES_desc-preproc_T1w.nii.gz'))
t1w = t1w_img.get_fdata()



In [64]:
## Generate streamlines for each bundle

for tract in bundle_dict.keys():
    
    temp_tract = bundle_dict[tract]['tractogram']
    temp_tract.to_rasmm()
    bundle_dict[tract]['t1w'] = transform_streamlines(temp_tract.streamlines,
                                np.linalg.inv(t1w_img.affine))

In [87]:
## Generate list of significant bundles based on model output

sig_bundles = model_output_df[(model_output_df.predictor=='led_sch_seda_s_mn_avg_eb')&(model_output_df['p_adj']<0.05)]

sig_bundles = sig_bundles.tractID.unique()
sig_bundles = [s.replace('_', '') for s in sig_bundles]


['ARCL', 'ARCR', 'CGCR', 'Motor', 'SupParietal', 'Temporal', 'pARCL']

In [88]:
## Create Color Map (overall bundles and model output)

color_dict = {tract: rgb_list[i] for i, tract in enumerate(tracts)}

mod_color_map = create_colormap(np.array(model_output_df[model_output_df.predictor=='led_sch_seda_s_mn_avg_eb']['neg_log10p_adj']), 'viridis')

full_color_map = rgb_list


corr_color_map_dict = {
    tract: mod_color_map[i]
    for i, tract in enumerate(tracts)
}

full_color_map_dict = {
    tract: full_color_map[i]
    for i, tract in enumerate(tracts)
}

## setup bundle actors for tracts with s
for tract in bundle_dict.keys():
    
    bundle_actor = lines_as_tubes(bundle_dict[tract]['t1w'], 8,
                                  colors=corr_color_map_dict[tract]
                                 )
    bundle_dict[tract]['bundle_actor'] = bundle_actor

In [92]:
# ## All bundles
scene = window.Scene()

for tract in bundle_dict.keys():
    
    scene.add(bundle_dict[tract]['bundle_actor'])

for slicer in slicers_z:
    scene.add(slicer)

## Figure 2A

In [93]:
## Significant bundles
scene = window.Scene()

for tract in bundle_dict.keys():
    
    if tract in sig_bundles:
        scene.add(bundle_dict[tract]['bundle_actor'])

for slicer in slicers:
    scene.add(slicer)

In [94]:
## Interactive view of bundle renderings
window.show(scene, size=(1200, 1200), reset_camera=False)


## Supplemental Figure 2

In [95]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np
from glob import glob

In [96]:
right_hemisphere = [tract for tract in bundle_dict.keys() if tract[-1]=='R']
left_hemisphere = [tract for tract in bundle_dict.keys() if tract[-1]=='L']
collosal = [tract for tract in bundle_dict.keys() if tract[-1]=='l']+['Motor']

In [97]:
for tract in bundle_dict.keys():
    
    
    bundle_actor = lines_as_tubes(bundle_dict[tract]['t1w'], 8,
                                  colors=color_dict[tract]
                                 )

    bundle_dict[tract]['bundle_actor'] = bundle_actor

In [99]:
## Generate Right Hemisphere Plots

for tract in right_hemisphere:
    
    scene = window.Scene()
    scene.add(bundle_dict[tract]['bundle_actor'])

    for slicer in slicers:
        scene.add(slicer)
        
    scene.set_camera(position=(-420.74, 148.69, 64.97),
                 focal_point=(96.32, 114.00, 96.00),
                 view_up=(-0.06, -0.02, 1.00))
    
    file_name = f'../figures/{tract}_plot.png'
    window.record(scene, out_path=file_name, size=(2400, 2400))


ARCR
ATRR
CGCR
CSTR
IFOR
ILFR
SLFR
UNCR
pARCR
VOFR


In [158]:
## Generate Left Hemisphere Plots
# Active Camera
#    Position (720.10, 137.09, 166.40)
#    Focal Point (96.32, 114.00, 96.00)
#    View Up (-0.11, -0.06, 0.99)


for tract in left_hemisphere:
    
    scene = window.Scene()
    scene.add(bundle_dict[tract]['bundle_actor'])

    for slicer in slicers:
        scene.add(slicer)
        
    scene.set_camera(position=(720.10, 137.09, 166.40),
                 focal_point=(96.32, 114.00, 96.00),
                 view_up=(-0.06, -0.02, 1.00))
    
    file_name = f'../figures/{tract}_plot.png'
    window.record(scene, out_path=file_name, size=(2400, 2400))

In [159]:
## Generate Collosal Plots
#  Position (96.00, 114.00, 599.59)
#    Focal Point (96.00, 114.00, 81.28)
#    View Up (0.00, 1.00, 0.00)

for tract in collosal:
    
    scene = window.Scene()
    scene.add(bundle_dict[tract]['bundle_actor'])

    for slicer in slicers_z:
        scene.add(slicer)
        
    scene.set_camera(position=(96.00, 114.00, 599.59),
                 focal_point=(96.00, 114.00, 81.28),
                 view_up=(0, 0, 1))
    
    file_name = f'../figures/{tract}_plot.png'
    window.record(scene, out_path=file_name, size=(2400, 2400))




In [4]:
bundle_order = ["VOFR","pARCR","ARCR","ATRR","CSTR","IFOR","ILFR","SLFR","UNCR","CGCR",
                "Orbital", "AntFrontal", "SupFrontal", "Motor","SupParietal", "PostParietal", "Temporal", "Occipital",
                "CGCL","UNCL","SLFL","ILFL","IFOL","CSTL","ATRL","ARCL","pARCL","VOFL"]

In [5]:
# Define a custom key function
def custom_key(item):
    # Find the first substring from sorting_substrings that appears in the item
    for substring in bundle_order:
        if substring in item:
            return bundle_order.index(substring)
    # If none of the substrings are found, return a default value
    return len(bundle_order)



['VOFR', 'pARCR', 'ARCR', 'ATRR', 'CSTR', 'IFOR', 'ILFR', 'SLFR', 'UNCR', 'CGCR', 'Orbital', 'AntFrontal', 'SupFrontal', 'Motor', 'SupParietal', 'PostParietal', 'Temporal', 'Occipital', 'CGCL', 'UNCL', 'SLFL', 'ILFL', 'IFOL', 'CSTL', 'ATRL', 'ARCL', 'pARCL', 'VOFL']


In [9]:

# Define the number of rows and columns in the montage
num_rows = 4  # Adjust as needed
num_cols = 7  # Adjust as needed

# Load your PNG images and store them in a list
image_paths = glob('../figures/*_plot.png')

# Define text captions for each image (adjust as needed)
captions = ['Right Vertical Occipital Fasciculus (VOF_R)', 'Right Posterior Arcuate (pARC_R)',
            'Right Arcuate (ARC_R)','Right Thalamic Radiation (ATR_R)', 'Right Corticospinal (CST_R)',
            'Right IFOF (IFOF_R)', 'Right ILF (ILF_R)', 'Right SLF (SLF_R)','Right Uncinate (UNC_R)',
            'Right Cingulum Cingulate (CGC_R)','Orbital','AntFrontal', 'Superior Frontal (SupFrontal)', 'Motor',
            'Superior Parietal (SupParietal)','Post Parietal', 'Temporal', 'Occipital', 
            'Left Cingulum Cingulate (CGC_L)','Left Uncinate (UNC_L)', 'Left SLF (SLF_L)','Left ILF (ILF_L)',
            'Left IFOF (IFOF_L)',  'Left Corticospinal (CST_L)','Left Thalamic Radiation (ATR_L)',
            'Left Arcuate (ARC_L)','Left Posterior Arcuate (pARC_L)', 'Left Vertical Occipital Fasciculus (VOF_L)']


image_paths = sorted(image_paths, key=custom_key)
img_cap_dict = {caption: {'path':image_paths[i],
                          'img': Image.open(image_paths[i])} for i, caption in enumerate(captions)}

# images = [Image.open(img_cap_dict[key]) for key in img_cap_dict.keys()]

# Calculate the size of the montage image
image_width, image_height = img_cap_dict['Right Vertical Occipital Fasciculus (VOF_R)']['img'].size
montage_width = image_width * num_cols
montage_height = image_height * num_rows

# # Create a blank montage image
montage = Image.new("RGB", (montage_width, montage_height))

# # Create a drawing context to add text captions
draw = ImageDraw.Draw(montage)

# # Load a font for captions (adjust the font file path and size as needed)
font = ImageFont.truetype("/Library/Fonts/Microsoft/Palatino Linotype.ttf", size=200)

# Maximum width for a line of text (adjust as needed)
max_text_width = image_width - 20

# Iterate through the images and paste them onto the montage with captions
for i,key in enumerate(img_cap_dict.keys()):
    
    print(img_cap_dict[key]['path'],key)
    
    row = i // num_cols
    col = i % num_cols
    x_offset = col * image_width
    y_offset = row * image_height
    montage.paste(img_cap_dict[key]['img'], (x_offset, y_offset))

    # Add the text caption above each image
    caption = key

    # Split the caption into multiple lines if it exceeds the maximum width
    lines = []
    line = ""
    for word in caption.split():
        test_line = line + " " + word if line else word
        test_width, _ = draw.textsize(test_line, font=font)
        if test_width <= max_text_width:
            line = test_line
        else:
            lines.append(line)
            line = word
    if line:
        lines.append(line)

    # Calculate the total height of the text block
    total_text_height = len(lines) * draw.textbbox((0, 0), lines[0], font=font)[3]

    # Calculate the starting y-position to center the text block above the image
    text_y = y_offset + 40

    # Draw each line of text centered within the image width
    for line in lines:
        text_bbox = draw.textbbox((0, 0), line, font=font)
        text_x = x_offset + (image_width - (text_bbox[2] - text_bbox[0])) // 2
        draw.text((text_x, text_y), line, fill="white", font=font)
        text_y += text_bbox[3] - text_bbox[1]

# # Save the montage image with captions
montage.save("montage_with_captions.png")

# Optionally, display the montage image
montage.show()

../figures/VOFR_plot.png Right Vertical Occipital Fasciculus (VOF_R)
../figures/pARCR_plot.png Right Posterior Arcuate (pARC_R)


/var/folders/5s/p7n2md3n27s89lm6qbs7996c0000gn/T/ipykernel_92997/3032877010.py:61: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  test_width, _ = draw.textsize(test_line, font=font)
/var/folders/5s/p7n2md3n27s89lm6qbs7996c0000gn/T/ipykernel_92997/3032877010.py:61: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  test_width, _ = draw.textsize(test_line, font=font)
/var/folders/5s/p7n2md3n27s89lm6qbs7996c0000gn/T/ipykernel_92997/3032877010.py:61: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  test_width, _ = draw.textsize(test_line, font=font)
/var/folders/5s/p7n2md3n27s89lm6qbs7996c0000gn/T/ipykernel_92997/3032877010.py:61: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  

../figures/ARCR_plot.png Right Arcuate (ARC_R)
../figures/ATRR_plot.png Right Thalamic Radiation (ATR_R)


/var/folders/5s/p7n2md3n27s89lm6qbs7996c0000gn/T/ipykernel_92997/3032877010.py:61: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  test_width, _ = draw.textsize(test_line, font=font)
/var/folders/5s/p7n2md3n27s89lm6qbs7996c0000gn/T/ipykernel_92997/3032877010.py:61: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  test_width, _ = draw.textsize(test_line, font=font)
/var/folders/5s/p7n2md3n27s89lm6qbs7996c0000gn/T/ipykernel_92997/3032877010.py:61: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  test_width, _ = draw.textsize(test_line, font=font)
/var/folders/5s/p7n2md3n27s89lm6qbs7996c0000gn/T/ipykernel_92997/3032877010.py:61: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  

../figures/CSTR_plot.png Right Corticospinal (CST_R)
../figures/IFOR_plot.png Right IFOF (IFOF_R)


...

/var/folders/5s/p7n2md3n27s89lm6qbs7996c0000gn/T/ipykernel_92997/3032877010.py:61: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  test_width, _ = draw.textsize(test_line, font=font)
/var/folders/5s/p7n2md3n27s89lm6qbs7996c0000gn/T/ipykernel_92997/3032877010.py:61: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  test_width, _ = draw.textsize(test_line, font=font)
/var/folders/5s/p7n2md3n27s89lm6qbs7996c0000gn/T/ipykernel_92997/3032877010.py:61: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  test_width, _ = draw.textsize(test_line, font=font)
/var/folders/5s/p7n2md3n27s89lm6qbs7996c0000gn/T/ipykernel_92997/3032877010.py:61: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  

../figures/ATRL_plot.png Left Thalamic Radiation (ATR_L)
../figures/ARCL_plot.png Left Arcuate (ARC_L)
../figures/pARCL_plot.png Left Posterior Arcuate (pARC_L)


/var/folders/5s/p7n2md3n27s89lm6qbs7996c0000gn/T/ipykernel_92997/3032877010.py:61: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  test_width, _ = draw.textsize(test_line, font=font)
/var/folders/5s/p7n2md3n27s89lm6qbs7996c0000gn/T/ipykernel_92997/3032877010.py:61: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  test_width, _ = draw.textsize(test_line, font=font)
/var/folders/5s/p7n2md3n27s89lm6qbs7996c0000gn/T/ipykernel_92997/3032877010.py:61: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  test_width, _ = draw.textsize(test_line, font=font)
/var/folders/5s/p7n2md3n27s89lm6qbs7996c0000gn/T/ipykernel_92997/3032877010.py:61: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  

../figures/VOFL_plot.png Left Vertical Occipital Fasciculus (VOF_L)


ValueError: unknown file extension: .svg